# Background

In [1]:
import platform
display(platform.system())
import os
file_download_link = 'https://data.ca.gov/dataset/3f08b68e-1d1a-4ba4-a07d-1ec3392ed191/resource/78a9d6ee-ec9a-4c25-ae34-5bac44010cb2/download/qcew_2016-2019.csv'
if os.name == 'nt':
    print('Please download your dataset here:', file_download_link)
else:
    # If on another OS, use wget to download the CSV file directly
    !wget -O qcew_2016-2019.csv "$file_download_link" -o /dev/null
    print("File downloaded successfully as qcew_2016-2019.csv")

'Linux'

File downloaded successfully as qcew_2016-2019.csv


In [2]:
#Only run if using Google Colab, do not run in VSCode
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 20.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('./qcew_2016-2019.csv')
print(df.shape)
df.head()

(1009955, 15)


,Area Type,Area Name,Year,Quarter,Ownership,NAICS Level,NAICS Code,Industry Name,Establishments,Average Monthly Employment,1st Month Emp,2nd Month Emp,3rd Month Emp,Total Wages (All Workers),Average Weekly Wages
0,County,Alameda County,2016,1st Qtr,Federal Government,2,1023,Financial Activities,1,10,10,10,10,359690.0,2767.0
1,County,Alameda County,2016,1st Qtr,Federal Government,2,1028,Public Administration,72,5174,5193,5143,5188,136068970.0,2023.0
2,County,Alameda County,2016,1st Qtr,Private,5,11121,Vegetable and Melon Farming,7,29,31,27,29,210934.0,560.0
3,County,Alameda County,2016,1st Qtr,Private,4,1114,Greenhouse and Nursery Production,7,52,61,64,31,748616.0,1107.0
4,County,Alameda County,2016,1st Qtr,Private,5,11142,Nursery and Floriculture Production,7,52,61,64,31,748616.0,1107.0


In [5]:
#Lots of data points, maybe we can drop the older years
print("Number of datapoints from 2018-2019: ", df.loc[df['Year'] == 2019].shape[0] + df.loc[df['Year'] == 2018].shape[0])

Number of datapoints from 2018-2019:  506913


In [6]:
#500K datapoints should be sufficient, and it should also be more accurate since its newer data
df = df[df['Year'] != 2016]
df = df[df['Year'] != 2017]
print(df.shape)

(506913, 15)


In [7]:
df.columns

Index(['Area Type', 'Area Name', 'Year', 'Quarter', 'Ownership', 'NAICS Level',
       'NAICS Code', 'Industry Name', 'Establishments',
       'Average Monthly Employment', '1st Month Emp', '2nd Month Emp',
       '3rd Month Emp', 'Total Wages (All Workers)', 'Average Weekly Wages'],
      dtype='object')

In [8]:
# Map NCAIS codes to higher level employment sectors
sector_dict = {
    11 : "Agriculture, Forestry, Fishing and Hunting",
    21 : "Mining, Quarrying, and Oil and Gas Extraction",
    22 : "Utilities",
    23 : "Construction",
    31 : "Manufacturing",
    32 : "Manufacturing",
    33 : "Manufacturing",
    42 : "Wholesale Trade",
    44 : "Retail Trade",
    45 : "Retail Trade",
    48 : "Transportation and Warehousing",
    49 : "Transportation and Warehousing",
    51 : "Information",
    52 : "Finance and Insurance",
    53 : "Real Estate and Rental and Leasing",
    54 : "Professional, Scientific, and Technical Services",
    55 : "Management of Companies and Enterprises",
    56 : "Administrative and Support and Waste Management and Remediation Services",
    61 : "Educational Services",
    62 : "Health Care and Social Assistance",
    71 : "Arts, Entertainment, and Recreation",
    72 : "Accommodation and Food Services",
    81 : "Other Services (except Public Administration)",
    92 : "Public Administration",
    928110 : "Military",
}

def map_sector(row):
    # Check first for military code
    if row['NAICS Code'] == 928110:
        return sector_dict.get(928110)
    # Otherwise use sector dictionary ... otherwise use 'Industry Name'
    return sector_dict.get(int(str(row['NAICS Code'])[:2]), row['Industry Name'])

df['Sector'] = df.apply(map_sector, axis=1)
df.head()

#Since we mapped industries and NCAIS codes to sectors, we can drop the lower level columns
df.drop(['NAICS Level','NAICS Code', 'Industry Name'],axis=1,inplace=True)
df.head()

,Area Type,Area Name,Year,Quarter,Ownership,Establishments,Average Monthly Employment,1st Month Emp,2nd Month Emp,3rd Month Emp,Total Wages (All Workers),Average Weekly Wages,Sector
147,County,Marin County,2019,Annual,Private,6,46,0,0,0,3170232.0,1340.0,Retail Trade
148,County,Marin County,2019,Annual,Private,10,819,0,0,0,28355846.0,666.0,Retail Trade
149,County,Marin County,2019,Annual,Private,19,189,0,0,0,10466322.0,1065.0,Administrative and Support and Waste Managemen...
150,County,Modoc County,2019,Annual,Private,4,19,0,0,0,1008116.0,1048.0,"Agriculture, Forestry, Fishing and Hunting"
151,County,Modoc County,2019,Annual,Private,4,19,0,0,0,1008116.0,1048.0,"Agriculture, Forestry, Fishing and Hunting"


In [9]:
#We don't need to know the total wages for all of the workers, who owns the business, or the individual months of employement
df.drop(['Ownership','1st Month Emp','2nd Month Emp','3rd Month Emp','Total Wages (All Workers)'],axis=1,inplace=True)
df.head()

,Area Type,Area Name,Year,Quarter,Establishments,Average Monthly Employment,Average Weekly Wages,Sector
147,County,Marin County,2019,Annual,6,46,1340.0,Retail Trade
148,County,Marin County,2019,Annual,10,819,666.0,Retail Trade
149,County,Marin County,2019,Annual,19,189,1065.0,Administrative and Support and Waste Managemen...
150,County,Modoc County,2019,Annual,4,19,1048.0,"Agriculture, Forestry, Fishing and Hunting"
151,County,Modoc County,2019,Annual,4,19,1048.0,"Agriculture, Forestry, Fishing and Hunting"


In [10]:
#Checking what Area Types we have
df['Area Type'].unique()

array(['County', 'California - Statewide', 'United States'], dtype=object)

In [12]:
#What is the distribution of the employment sectors?
df['Sector'].value_counts()

,count
Sector,
Manufacturing,61090
Retail Trade,59393
Health Care and Social Assistance,37066
Wholesale Trade,35930
Other Services (except Public Administration),33813
Public Administration,33618
"Professional, Scientific, and Technical Services",31044
Construction,30641
Transportation and Warehousing,25439


In [13]:
#We want to focus on the county level, lets ensure there are enough datapoints to drop the rest
print(df.loc[df['Area Type'] == 'County'].shape)
df = df[df['Area Type'] == 'County']

(441540, 8)


In [14]:
#Since there may be overlap between Quarters or the Annual datapoints, lets only use Annual
print(df.loc[df['Quarter'] == 'Annual'].shape)
df = df[df['Quarter'] == 'Annual']

(89255, 8)


In [15]:
#Now we can drop 'Area Type' and 'Quarter'
df.drop(['Area Type','Quarter'],axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Area Name,Year,Establishments,Average Monthly Employment,Average Weekly Wages,Sector
0,Marin County,2019,6,46,1340.0,Retail Trade
1,Marin County,2019,10,819,666.0,Retail Trade
2,Marin County,2019,19,189,1065.0,Administrative and Support and Waste Managemen...
3,Modoc County,2019,4,19,1048.0,"Agriculture, Forestry, Fishing and Hunting"
4,Modoc County,2019,4,19,1048.0,"Agriculture, Forestry, Fishing and Hunting"


In [16]:
df.shape

(89255, 6)

In [17]:
#Check for missing values
df.isnull().sum()

,0
Area Name,0
Year,0
Establishments,0
Average Monthly Employment,0
Average Weekly Wages,0
Sector,0


In [18]:
#Drop outliers using IQR
def dropOutliers(df, colName):
    [Q1, Q2, Q3] = df[colName].quantile([0.25, 0.5, 0.75])
    IQR = Q3 - Q1
    return df[(df[colName] >= (Q1 - (1.5 * IQR))) & (df[colName] <= (Q3 + (1.5 * IQR)))]
df = dropOutliers(df, 'Establishments')
df = dropOutliers(df, 'Average Monthly Employment')
df = dropOutliers(df, 'Average Weekly Wages')
print(df.shape)

(65689, 6)
